# Exploring UV extinction curves – Part 1

## Learning Goals

By the end of this tutorial, you will be able to:

- Understand what a UV extinction curve is and how it is used to study dust properties
- Access data from the MAST archives directly from Python
- Perform data querys by Target Name and narrow them down to the desired observation
- Save arrays onto _.txt_ files
- Implement Python functions

## Introduction

**Extinction curve background**: An extinction curve represents the wavelength dependence of dust extinction. It compares the inherent Spectral Energy Distribution (SED) (~dust-free) of a star to the observed SED affected by dust extinction. Normally these curves are created by representing $k(\lambda-V)$ versus $1/\lambda$, with $\lambda$ being the wavelength.

Extinction is relevant in many different scenarios. Dust can be found very near the observed object – such as stars having disks or proto-stellar clouds surrounding it – or can be far away from it but still affect the observation performed – for example dust in a galaxy that happens to be between the observer and the star.

**Defining some terms**:

- **Color index**: difference between magnitude of a star in 2 different passbands, typically B and V. Symbol: $(B-V)$. 
- **Extinction**: measure of interstellar reddening quantified by the difference in magnitudes. Symbol: $A_\lambda$.
- **Spectral type**: stellar classification from hotter (O stars) to cooler (M stars). Temperature defines a star's "color" and surface brightness.


**Useful equations**
- $k(\lambda-V) = \frac{E(\lambda-V)}{E(B-V)} = \frac{m(\lambda-V)-m(\lambda-V)_o}{(B-V)-(B-V)_o} = \frac{A(\lambda)-A(V)}{A(B)-A(V)}$

    _**Note**: the $x_o$ terms refer to the star that is nearly unaffected by dust, i.e. $(B-V)$ corresponds to the observed color index and $(B-V)_o$ to the observed color index if there was no extinction due to dust. The stars should have the same spectral type in order to perform this comparison._

## Imports 

The first step will be to import the libraries we will be using throughout this tutorial: 

- _Observations_ from _astroquery.mast_ to query the Barbara A. Mikulski Archive for Space Telescopes (MAST).
- _fits_ from _astropy.io_ for accessing FITS files
- _numpy_ for array manipulations

In [1]:
from astroquery.mast import Observations
from astropy.io import fits
import numpy as np

_**Warning**: If you have not installed the astroquery package in your computer, you may need to. Information about astroquery can be found <a href="https://astroquery.readthedocs.io/en/latest/">here</a>._

## Loading data

The next step is to find the data file we will use. This is similar to searching through the <a href="https://mast.stsci.edu/portal/Mashup/Clients/Mast/Portal.html">MAST portal</a> since we will be using specific keywords to find the file. We will be using the _Observations_ from _astroquery.mast_ to download UV data directly from the _IUE_ archive.


The target name of the first star we will be studying is AzV 214 (observation ID: swp22372), collected by International Ultraviolet Explorer (IUE). This star is known to be reddened by dust. **Include brief explanation of why this star is reddened**.

To get the data products we want we need the observation ID corresponding to it, in order to do this we can first query the collection of IUE using _Observations_ by using the target name as shown below.

_**Heads-up:** in the database this object is labeled AV214 instead of AzV 214._

In [2]:
IUEobs_av214 = Observations.query_criteria(objectname="AV214",obs_collection="IUE")
data_products_av214 = Observations.get_product_list(IUEobs_av214)

data_products_av214

obsID,obs_collection,dataproduct_type,obs_id,description,type,dataURI,productType,productGroupDescription,productSubGroupDescription,productDocumentationURL,project,prvversion,proposal_id,productFilename,size,parent_obsid,dataRights,calib_level
str6,str3,str8,str8,str56,str1,str62,str9,str28,str56,str1,str1,str1,str5,str20,int64,str6,str6,int64
275789,IUE,spectrum,lwp02621,ELBLL,S,mast:IUE/url/pub/iue/data/lwp/02000/lwp02621.elbll.gz,AUXILIARY,--,--,--,--,--,FM034,lwp02621.elbll.gz,95042,275789,PUBLIC,2
275789,IUE,spectrum,lwp02621,LILO,S,mast:IUE/url/pub/iue/data/lwp/02000/lwp02621.lilo.gz,AUXILIARY,--,--,--,--,--,FM034,lwp02621.lilo.gz,490665,275789,PUBLIC,2
275789,IUE,spectrum,lwp02621,MELOL,S,mast:IUE/url/pub/iue/data/lwp/02000/lwp02621.melol.gz,AUXILIARY,--,--,--,--,--,FM034,lwp02621.melol.gz,12028,275789,PUBLIC,2
275789,IUE,spectrum,lwp02621,RAW,S,mast:IUE/url/pub/iue/data/lwp/02000/lwp02621.raw.gz,AUXILIARY,--,--,--,--,--,FM034,lwp02621.raw.gz,329022,275789,PUBLIC,2
275789,IUE,spectrum,lwp02621,RILO,S,mast:IUE/url/pub/iue/data/lwp/02000/lwp02621.rilo.gz,AUXILIARY,--,--,--,--,--,FM034,lwp02621.rilo.gz,334246,275789,PUBLIC,2
275789,IUE,spectrum,lwp02621,SILO,S,mast:IUE/url/pub/iue/data/lwp/02000/lwp02621.silo.gz,AUXILIARY,--,--,--,--,--,FM034,lwp02621.silo.gz,85947,275789,PUBLIC,2
275789,IUE,spectrum,lwp02621,Preview-Full,S,mast:IUE/url/browse/previews/iue/mx/lwp/02000/gif/lwp02621.gif,PREVIEW,--,--,--,--,--,FM034,lwp02621.gif,7103,275789,PUBLIC,2
275789,IUE,spectrum,lwp02621,MXLO,S,mast:IUE/url/pub/iue/data/lwp/02000/lwp02621.mxlo.gz,SCIENCE,Minimum Recommended Products,--,--,--,--,FM034,lwp02621.mxlo.gz,17736,275789,PUBLIC,2
275789,IUE,spectrum,lwp02621,(extracted spectra/vo spectral container/SSAP) fits file,S,mast:IUE/url/pub/vospectra/iue2/lwp02621mxlo_vo.fits,SCIENCE,Minimum Recommended Products,(extracted spectra/vo spectral container/SSAP) fits file,--,--,--,FM034,lwp02621mxlo_vo.fits,48960,275789,PUBLIC,2


Two different observations, "swp22372" for the short wavelength domain and "lwr17263" for the long wave domain, have been chosen in order to provide a more complete understanding of the extinction behaviour over different wavelengths. For this tutorial we have chosen these specific observations, but the reader is welcome to use any other of the IDs displayed on the table below.  It should be taken into account that some of the data may need some prior calibration before using it. 

- Observation ID: "swp22372":

In [3]:
data_products_av214[1144]

obsID,obs_collection,dataproduct_type,obs_id,description,type,dataURI,productType,productGroupDescription,productSubGroupDescription,productDocumentationURL,project,prvversion,proposal_id,productFilename,size,parent_obsid,dataRights,calib_level
str6,str3,str8,str8,str56,str1,str62,str9,str28,str56,str1,str1,str1,str5,str20,int64,str6,str6,int64
318326,IUE,spectrum,swp22372,(extracted spectra/vo spectral container/SSAP) fits file,S,mast:IUE/url/pub/vospectra/iue2/swp22372mxlo_vo.fits,SCIENCE,Minimum Recommended Products,(extracted spectra/vo spectral container/SSAP) fits file,--,--,--,FM197,swp22372mxlo_vo.fits,48960,318326,PUBLIC,2


In [4]:
IUEobs_av214_sw = Observations.query_criteria(obs_collection="IUE", obs_id="swp22372")
data_products_av214_sw = Observations.get_product_list(IUEobs_av214_sw)
yourProd_av214_sw = Observations.filter_products(data_products_av214_sw, extension='swp22372mxlo_vo.fits')

yourProd_av214_sw

obsID,obs_collection,dataproduct_type,obs_id,description,type,dataURI,productType,productGroupDescription,productSubGroupDescription,productDocumentationURL,project,prvversion,proposal_id,productFilename,size,parent_obsid,dataRights,calib_level
str6,str3,str8,str8,str56,str1,str62,str9,str28,str56,str1,str1,str1,str5,str20,int64,str6,str6,int64
318326,IUE,spectrum,swp22372,(extracted spectra/vo spectral container/SSAP) fits file,S,mast:IUE/url/pub/vospectra/iue2/swp22372mxlo_vo.fits,SCIENCE,Minimum Recommended Products,(extracted spectra/vo spectral container/SSAP) fits file,--,--,--,FM197,swp22372mxlo_vo.fits,48960,318326,PUBLIC,2


Now that we have the data file, we can download it by using the `download_products` function:

In [5]:
Observations.download_products(yourProd_av214_sw, mrp_only = False, cache = False) 

Local Path,Status,Message,URL
str48,str8,object,object
./mastDownload/IUE/swp22372/swp22372mxlo_vo.fits,COMPLETE,None,None


If the download does not happen automatically, by visiting the URL displayed by the `download_products` method you will be able to download the file. 

As stated before, in order to have a more complete image of our target, let's download a different observation for the same object, but in a different wavelength range, we can use the following observation ID: lwr17263 (this is still AzV 214).

- Observation ID: "lwr17263":

In [6]:
data_products_av214[923]

obsID,obs_collection,dataproduct_type,obs_id,description,type,dataURI,productType,productGroupDescription,productSubGroupDescription,productDocumentationURL,project,prvversion,proposal_id,productFilename,size,parent_obsid,dataRights,calib_level
str6,str3,str8,str8,str56,str1,str62,str9,str28,str56,str1,str1,str1,str5,str20,int64,str6,str6,int64
305282,IUE,spectrum,lwr17263,(extracted spectra/vo spectral container/SSAP) fits file,S,mast:IUE/url/pub/vospectra/iue2/lwr17263mxlo_vo.fits,SCIENCE,Minimum Recommended Products,(extracted spectra/vo spectral container/SSAP) fits file,--,--,--,FM197,lwr17263mxlo_vo.fits,48960,305282,PUBLIC,2


In [7]:
IUEobs_av214_lw = Observations.query_criteria(obs_collection="IUE", obs_id="lwr17263")
data_products_av214_lw = Observations.get_product_list(IUEobs_av214_lw)
yourProd_av214_lw = Observations.filter_products(data_products_av214_lw, extension='lwr17263mxlo_vo.fits')
Observations.download_products(yourProd_av214_lw, mrp_only = False, cache = False) 

Local Path,Status,Message,URL
str48,str8,object,object
./mastDownload/IUE/lwr17263/lwr17263mxlo_vo.fits,COMPLETE,None,None


Now let's do the same for the unreddened star. The target name of this star is AzV 380 (observation IDs chosen: swp10319, lwr17265) and was also collected by IUE.

In [8]:
IUEobs_av380 = Observations.query_criteria(objectname="AV380",obs_collection="IUE")
data_products_av380 = Observations.get_product_list(IUEobs_av380)

data_products_av380

obsID,obs_collection,dataproduct_type,obs_id,description,type,dataURI,productType,productGroupDescription,productSubGroupDescription,productDocumentationURL,project,prvversion,proposal_id,productFilename,size,parent_obsid,dataRights,calib_level
str6,str3,str8,str8,str56,str1,str62,str9,str28,str56,str1,str1,str1,str5,str20,int64,str6,str6,int64
282255,IUE,spectrum,lwp12438,ELBLL,S,mast:IUE/url/pub/iue/data/lwp/12000/lwp12438.elbll.gz,AUXILIARY,--,--,--,--,--,MCJBB,lwp12438.elbll.gz,181294,282255,PUBLIC,2
282255,IUE,spectrum,lwp12438,LILO,S,mast:IUE/url/pub/iue/data/lwp/12000/lwp12438.lilo.gz,AUXILIARY,--,--,--,--,--,MCJBB,lwp12438.lilo.gz,505184,282255,PUBLIC,2
282255,IUE,spectrum,lwp12438,MELOL,S,mast:IUE/url/pub/iue/data/lwp/12000/lwp12438.melol.gz,AUXILIARY,--,--,--,--,--,MCJBB,lwp12438.melol.gz,12305,282255,PUBLIC,2
282255,IUE,spectrum,lwp12438,RAW,S,mast:IUE/url/pub/iue/data/lwp/12000/lwp12438.raw.gz,AUXILIARY,--,--,--,--,--,MCJBB,lwp12438.raw.gz,339065,282255,PUBLIC,2
282255,IUE,spectrum,lwp12438,RILO,S,mast:IUE/url/pub/iue/data/lwp/12000/lwp12438.rilo.gz,AUXILIARY,--,--,--,--,--,MCJBB,lwp12438.rilo.gz,344146,282255,PUBLIC,2
282255,IUE,spectrum,lwp12438,SILO,S,mast:IUE/url/pub/iue/data/lwp/12000/lwp12438.silo.gz,AUXILIARY,--,--,--,--,--,MCJBB,lwp12438.silo.gz,87776,282255,PUBLIC,2
282255,IUE,spectrum,lwp12438,Preview-Full,S,mast:IUE/url/browse/previews/iue/mx/lwp/12000/gif/lwp12438.gif,PREVIEW,--,--,--,--,--,MCJBB,lwp12438.gif,8416,282255,PUBLIC,2
282255,IUE,spectrum,lwp12438,MXLO,S,mast:IUE/url/pub/iue/data/lwp/12000/lwp12438.mxlo.gz,SCIENCE,Minimum Recommended Products,--,--,--,--,MCJBB,lwp12438.mxlo.gz,17967,282255,PUBLIC,2
282255,IUE,spectrum,lwp12438,(extracted spectra/vo spectral container/SSAP) fits file,S,mast:IUE/url/pub/vospectra/iue2/lwp12438mxlo_vo.fits,SCIENCE,Minimum Recommended Products,(extracted spectra/vo spectral container/SSAP) fits file,--,--,--,MCJBB,lwp12438mxlo_vo.fits,48960,282255,PUBLIC,2


- Observation ID: "swp10319":

In [9]:
data_products_av380[26]

obsID,obs_collection,dataproduct_type,obs_id,description,type,dataURI,productType,productGroupDescription,productSubGroupDescription,productDocumentationURL,project,prvversion,proposal_id,productFilename,size,parent_obsid,dataRights,calib_level
str6,str3,str8,str8,str56,str1,str62,str9,str28,str56,str1,str1,str1,str5,str20,int64,str6,str6,int64
311430,IUE,spectrum,swp10319,(extracted spectra/vo spectral container/SSAP) fits file,S,mast:IUE/url/pub/vospectra/iue2/swp10319mxlo_vo.fits,SCIENCE,Minimum Recommended Products,(extracted spectra/vo spectral container/SSAP) fits file,--,--,--,HSCSP,swp10319mxlo_vo.fits,48960,311430,PUBLIC,2


In [10]:
IUEobs_av380_sw = Observations.query_criteria(obs_collection="IUE", obs_id="swp10319")
data_products_av380_sw = Observations.get_product_list(IUEobs_av380_sw)
yourProd_av380_sw = Observations.filter_products(data_products_av380_sw, extension='swp10319mxlo_vo.fits')
Observations.download_products(yourProd_av380_sw, mrp_only = False, cache = False) 

Local Path,Status,Message,URL
str48,str8,object,object
./mastDownload/IUE/swp10319/swp10319mxlo_vo.fits,COMPLETE,None,None


- Observation ID: "lwr17265":

In [11]:
data_products_av380[17]

obsID,obs_collection,dataproduct_type,obs_id,description,type,dataURI,productType,productGroupDescription,productSubGroupDescription,productDocumentationURL,project,prvversion,proposal_id,productFilename,size,parent_obsid,dataRights,calib_level
str6,str3,str8,str8,str56,str1,str62,str9,str28,str56,str1,str1,str1,str5,str20,int64,str6,str6,int64
305284,IUE,spectrum,lwr17265,(extracted spectra/vo spectral container/SSAP) fits file,S,mast:IUE/url/pub/vospectra/iue2/lwr17265mxlo_vo.fits,SCIENCE,Minimum Recommended Products,(extracted spectra/vo spectral container/SSAP) fits file,--,--,--,FM197,lwr17265mxlo_vo.fits,48960,305284,PUBLIC,2


In [12]:
IUEobs_av380_lw = Observations.query_criteria(obs_collection="IUE", obs_id="lwr17265")
data_products_av380_lw = Observations.get_product_list(IUEobs_av380_lw)
yourProd_av380_lw = Observations.filter_products(data_products_av380_lw, extension='lwr17265mxlo_vo.fits')
Observations.download_products(yourProd_av380_lw, mrp_only = False, cache = False) 

Local Path,Status,Message,URL
str48,str8,object,object
./mastDownload/IUE/lwr17265/lwr17265mxlo_vo.fits,COMPLETE,None,None


Again, if the download does not happen automatically, by visiting the URLs provided by `download_products` you will be able to download the necessary data to continue this tutorial.

## Exploring the downloaded data files

Now, let's explore the FITS file that we got for the reddened star. The paths to the files should correspond to wherever these files are located on your local computer, here the provided path is the one automatically generated by the `download_products` function. Feel free to change the filepath if you have stored the files in a different directory.

In [13]:
filename_av214_sw = "./mastDownload/IUE/swp22372/swp22372mxlo_vo.fits"
fits.info(filename_av214_sw)

Filename: ./mastDownload/IUE/swp22372/swp22372mxlo_vo.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU     350   ()      
  1  Spectral Container    1 BinTableHDU    141   1R x 4C   [495E, 495E, 495E, 495I]   


- **No. 0 (PRIMARY)**: This HDU contains meta-data related to the entire file.
- **No. 1 (Spectral Container)**: This HDU contains the spectral profile of the target as a function of wavelength.

The header of the file contains additional information about the data, in rows 54-61 important header information can be found, such as the wavelength reference frame or the reference equinox. It can be accessed in the following way (only some part of the information contained in the header is printed in this section, the user is encouraged to print the complete header if they are interested to see the whole information contained in it).  

In [14]:
with fits.open(filename_av214_sw) as hdulist: 
    header_av214_sw = hdulist[1].header

print(repr(header_av214_sw[54:61]))

EQUINOX =              2000.00 /                                                
RADECSYS= 'FK4'                /                                                
TIMESYS = 'UTC'                / Time system                                    
MJDREF  =              0.00000 / [d] MJD zero point for times                   
SPECSYS = 'TOPOCENT'           / wavelength reference frame                     
VOCLASS = 'SPECTRUM V1.0'      / VO Data Model                                  
VOSEGT  = 'SPECTRUM'           / Segment type                                   


Additionally, the header tells us information about what each column of the data contained in it represents:

In [15]:
print(repr(header_av214_sw[9:15]))

COMMENT  *** Column names ***                                                   
COMMENT                                                                         
TTYPE1  = 'WAVE    '           /                                                
TTYPE2  = 'FLUX    '           /                                                
TTYPE3  = 'SIGMA   '           /                                                
TTYPE4  = 'QUALITY '           /                                                


From this we know that the first column corresponds to the wavelengths, and the second column to the fluxes. The header also has information regarding the unit system used:

In [16]:
print(repr(header_av214_sw[23:28]))

COMMENT   *** Column units ***                                                  
COMMENT                                                                         
TUNIT1  = 'angstrom'           / wavelength unit is Angstrom                    
TUNIT2  = 'erg/cm**2/s/angstrom' / flux unit is ergs/cm2/sec/A                  
TUNIT3  = 'erg/cm**2/s/angstrom' / sigma unit is ergs/cm2/sec/A                 


## Data access and manipulation

The data contained in this fits failed can be accessed using io.fits and the `data` attribute:

In [17]:
with fits.open(filename_av214_sw) as hdulist:
    spectrum_av214_sw = hdulist[1].data

wav_av214_sw = spectrum_av214_sw[0][0] # angstrom, A
flux_av214_sw = spectrum_av214_sw[0][1] # ergs/cm2/sec/A

filename_av214_lw = "./mastDownload/IUE/lwr17263/lwr17263mxlo_vo.fits"
with fits.open(filename_av214_lw) as hdulist:
    spectrum_av214_lw = hdulist[1].data

wav_av214_lw = spectrum_av214_lw[0][0] # A
flux_av214_lw = spectrum_av214_lw[0][1] # ergs/cm2/sec/A

_**Heads-up**: some values of the flux may be contaminated or altered, and when applying the logarithm may result in NaN (Not a Number), in order to not have to deal with error messages throughout the notebook, we will use the `changeNaN` function displayed below to locate and change these problematic values. Since we will need to combine the reddened and un-reddened stars later on, they need to have the same length, so we will examine them together for problematic values._

An example of the problematic values can be seen in _flux_av214_lw_:

In [18]:
print("flux_av214_lw:", flux_av214_lw[-5:-3])
print("log(flux_av214_lw):", np.log10(flux_av214_lw[-5:-3]))

flux_av214_lw: [-2.2695733e-14 -2.0336736e-14]
log(flux_av214_lw): [nan nan]


<ipython-input-18-b266a8ce1588>:2: RuntimeWarning: invalid value encountered in log10
  print("log(flux_av214_lw):", np.log10(flux_av214_lw[-5:-3]))


In [19]:
def changeNaN(wavv,fluxx,wavv1,fluxx1):
    
    """
    Removes negative flux values to avoid encountering NaN when converting to flux scale
    
    Inputs:
    :param wavv,wavv1: input wavelength arrays
    :type wavv,wavv1: float array
    :param fluxx,fluxx1: input wavelength arrays
    :type fluxx,fluxx1: float array
    
    Outputs: 
    :return: wavv,fluxx,wavv1,fluxx1 without the corresponding problematic values
    :rtype: float array
    """
    
    prob = np.where(fluxx < 0)[0]
    wavv = np.delete(wavv,prob)
    fluxx = np.delete(fluxx,prob)
    wavv1 = np.delete(wavv1,prob)
    fluxx1 = np.delete(fluxx1,prob)
    
    prob2 = np.where(fluxx1 < 0)[0]
    wavv = np.delete(wavv,prob2)
    fluxx = np.delete(fluxx,prob2)
    wavv1 = np.delete(wavv1,prob2)
    fluxx1 = np.delete(fluxx1,prob2)
    
    return wavv, fluxx, wavv1, fluxx1

Let's do the same for the other files, use the `changeNaN` function to remove the problematic values.

In [20]:
filename_av380_sw = "./mastDownload/IUE/swp10319/swp10319mxlo_vo.fits"
with fits.open(filename_av380_sw) as hdulist:
    spectrum_av380_sw = hdulist[1].data

wav_av380_sw = spectrum_av380_sw[0][0] # A
flux_av380_sw = spectrum_av380_sw[0][1] # ergs/cm2/sec/A

filename_av380_lw = "./mastDownload/IUE/lwr17265/lwr17265mxlo_vo.fits"
with fits.open(filename_av380_lw) as hdulist:
    spectrum_av380_lw = hdulist[1].data

wav_av380_lw = spectrum_av380_lw[0][0] # A
flux_av380_lw = spectrum_av380_lw[0][1] # ergs/cm2/sec/A

# Remove problematic values
wav_av214_sw, flux_av214_sw, wav_av380_sw, flux_av380_sw = changeNaN(wav_av214_sw, flux_av214_sw, wav_av380_sw, flux_av380_sw)
wav_av214_lw, flux_av214_lw, wav_av380_lw, flux_av380_lw = changeNaN(wav_av214_lw, flux_av214_lw, wav_av380_lw, flux_av380_lw)

## Save values into your local computer

Now that we have removed the problematic values from our observations, let's explore how we can save them to our computers. To do this, we can use the function `saveArray` presented below:

In [21]:
def saveArray(array,filename):
    
    """
    Saves arrays into .txt files in current directory and checks that the files have been written properly
    
    Inputs:
    :param array: array to be saved to local directory
    :type array: float array
    :param filename: name of file to be saved
    :type filename: string
    
    Outputs: 
    :return: none
    :rtype: none
    """
    
    # First display the start and ending of the array to check later
    print('First 3 elements of original array:\n', array[:3])
    print('Last 3 elements of original array:\n', array[-3:])

    # Save the array in a text file
    np.savetxt(filename,array)
    
    # Display the saved content to check it was correctly written to the .txt file
    content = np.loadtxt(filename)
    print('First 3 elements of '+filename+':\n', content[:3])
    print('Last 3 elements of '+filename+':\n', content[-3:],'\n')
    
    return None

In [22]:
arrays = [wav_av214_sw, flux_av214_sw, wav_av380_sw, flux_av380_sw, wav_av214_lw, flux_av214_lw, wav_av380_lw, flux_av380_lw]
filenames = ['wav_av214_sw.txt', 'flux_av214_sw.txt', 'wav_av380_sw.txt', 'flux_av380_sw.txt', 'wav_av214_lw.txt', 'flux_av214_lw.txt', 'wav_av380_lw.txt', 'flux_av380_lw.txt']

for i in range(len(arrays)):
    saveArray(arrays[i],filenames[i])

First 3 elements of original array:
 [1150.5802 1152.2566 1153.933 ]
Last 3 elements of original array:
 [1975.3384 1977.0146 1978.691 ]
First 3 elements of wav_av214_sw.txt:
 [1150.5802002 1152.2565918 1153.9329834]
Last 3 elements of wav_av214_sw.txt:
 [1975.33837891 1977.01464844 1978.69104004] 

First 3 elements of original array:
 [1.1806722e-13 1.0497821e-13 9.7880498e-14]
Last 3 elements of original array:
 [9.0072332e-14 8.9639925e-14 9.0954899e-14]
First 3 elements of flux_av214_sw.txt:
 [1.18067218e-13 1.04978207e-13 9.78804982e-14]
Last 3 elements of flux_av214_sw.txt:
 [9.00723316e-14 8.96399247e-14 9.09548993e-14] 

First 3 elements of original array:
 [1150.5802 1152.2566 1153.933 ]
Last 3 elements of original array:
 [1975.3384 1977.0146 1978.691 ]
First 3 elements of wav_av380_sw.txt:
 [1150.5802002 1152.2565918 1153.9329834]
Last 3 elements of wav_av380_sw.txt:
 [1975.33837891 1977.01464844 1978.69104004] 

First 3 elements of original array:
 [2.0615980e-13 2.2626303e

## Exercises

Now you can try to do it yourself! Try to obtain the extinction curve of a Large Magellanic Cloud (LMC) following the steps presented for the SMC one and look for the differences in the trend between the two.

### 1. Load the data

The targets for this exercise will be:

- Sk -69 206 (Observation IDs: swp36552, lwp15751) (reddened star)
- Sk -67 5 (Observation IDs: swp04827, lwr04170) (unreddened star)

First, do the database search for the reddened star:

In [23]:
# You can copy, paste and modify the code corresponding to this part that we used for the SMC cloud here
# You can also use any function that was defined before


Narrow down the possibilities to the desired observation, observation ID: "swp36552":

In [24]:
# You can copy, paste and modify the code corresponding to this part that we used for the SMC cloud here
# You can also use any function that was defined before


Now, download the data as before:

In [25]:
# You can copy, paste and modify the code corresponding to this part that we used for the SMC cloud here
# You can also use any function that was defined before


And again, in order to cover a wider wavelength range, let's also download the other part of the spectra using the second observation ID provided, observation ID: "lwp15751":

In [26]:
# You can copy, paste and modify the code corresponding to this part that we used for the SMC cloud here
# You can also use any function that was defined before


Now, let's do the same for the unreddened star:

In [27]:
# You can copy, paste and modify the code corresponding to this part that we used for the SMC cloud here
# You can also use any function that was defined before


- Observation ID: "swp04827":

In [28]:
# You can copy, paste and modify the code corresponding to this part that we used for the SMC cloud here
# You can also use any function that was defined before


In [29]:
# You can copy, paste and modify the code corresponding to this part that we used for the SMC cloud here
# You can also use any function that was defined before


- Observation ID: "lwr04170"

In [30]:
# You can copy, paste and modify the code corresponding to this part that we used for the SMC cloud here
# You can also use any function that was defined before


In [31]:
# You can copy, paste and modify the code corresponding to this part that we used for the SMC cloud here
# You can also use any function that was defined before


### 2. Open the fits file and explore it

In [32]:
# You can copy, paste and modify the code corresponding to this part that we used for the SMC cloud here
# You can also use any function that was defined before


### 3. Save the data to your local computer

In [33]:
# You can copy, paste and modify the code corresponding to this part that we used for the SMC cloud here
# You can also use any function that was defined before


## Additional Resources

For more information about the MAST archive and details about mission data: 

<a href="https://mast.stsci.edu/api/v0/index.html">MAST API</a> <br>
<a href="https://archive.stsci.edu/iue/">International Ultraviolet Explorer Page (MAST)</a> <br>
<a href="https://archive.stsci.edu/iue/mdr_help.html">IUE Data Retrieval Help Page</a> 

For more information about extinction curves and their parametrization:


<a href="https://ui.adsabs.harvard.edu/abs/2003ApJ...594..279G/abstract">A quantitative comparison of SMC, LMC and Milky Way UV to NIR extinction curves</a> <br>
<a href="https://adsabs.harvard.edu/pdf/1990ApJS...72..163F">An analysis of the shapes of the ultraviolet extinction curves. III. An atlas of ultraviolet extinction curves</a> 

## About this Notebook

**Author**: Clara Puerto Sánchez <br>
**Updated on**: 06/07/2022

## Citations

If you use `astropy`, `numpy` or `scipy` for published research, please cite the
authors. Follow these links for more information about citing them:

* [Citing `astropy`](https://www.astropy.org/acknowledging.html)
* [Citing `numpy`](https://numpy.org/citing-numpy/)
* [Citing `scipy`](https://scipy.org/citing-scipy/)

<img style="float: right;" src="https://raw.githubusercontent.com/spacetelescope/notebooks/master/assets/stsci_pri_combo_mark_horizonal_white_bkgd.png" alt="Space Telescope Logo" width="200px"/>